## Imports

In [16]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [17]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 19th Jan. 26th


In [18]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [19]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [20]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [21]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [22]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-22-2025,Condo/Co-op,10302 Myrtlewood Cir W,Palm Beach Gardens,FL,33418.0,340000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/1...,Beaches MLS,RX-11044786,N,Y,26.873428,-80.106892


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [23]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [24]:
df2 = df

In [25]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-24    17
2025-01-22    16
2025-01-23    16
2025-01-21    12
2025-01-20     2
2025-01-27     1
Name: count, dtype: int64

In [26]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [27]:
df_top_ten = df2.head(10)

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
38,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Beaches MLS,RX-11028563,N,Y,26.918407,-80.066545
5,PAST SALE,2025-01-24,Condo/Co-op,140 SE 5th Ave #250,Boca Raton,FL,33432.0,1510000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Beaches MLS,RX-11050978,N,Y,26.348269,-80.076619
22,PAST SALE,2025-01-24,Condo/Co-op,550 S Ocean Blvd #1006,Boca Raton,FL,33432.0,1450000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11031947,N,Y,26.344431,-80.070692
64,PAST SALE,2025-01-21,Condo/Co-op,6 Little Pond Rd,Manalapan,FL,33462.0,1220000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Beaches MLS,RX-10948720,N,Y,26.570376,-80.043908
24,PAST SALE,2025-01-24,Condo/Co-op,424 Bay Colony Dr N,Juno Beach,FL,33408.0,1200000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Beaches MLS,RX-11034043,N,Y,26.860900,-80.065793
40,PAST SALE,2025-01-24,Condo/Co-op,3000 S Ocean Blvd #405,Boca Raton,FL,33432.0,1007500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Beaches MLS,RX-11024057,N,Y,26.321745,-80.075232
62,PAST SALE,2025-01-24,Condo/Co-op,242 Tresana Blvd #74,Jupiter,FL,33478.0,885000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Beaches MLS,RX-10959380,N,Y,26.926899,-80.155641
12,PAST SALE,2025-01-23,Condo/Co-op,3940 Back Bay 241 Dr #241,Jupiter,FL,33477.0,835000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/3940-Back-Ba...,Beaches MLS,RX-11044736,N,Y,26.924386,-80.092082
10,PAST SALE,2025-01-24,Condo/Co-op,7233 Promenade Dr #701,Boca Raton,FL,33433.0,745000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/7233-Prom...,MARMLS,A11697582,N,Y,26.334070,-80.157913
41,PAST SALE,2025-01-21,Condo/Co-op,5214 Lake Catalina Dr N Unit C,Boca Raton,FL,33496.0,720000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/5214-Lake...,Beaches MLS,RX-11014143,N,Y,26.414444,-80.126160


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [29]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [30]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [31]:
response_df = pd.DataFrame(response_list)

In [32]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [33]:
df_top_ten = merged_df

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,...,-80.066545,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Michael Montalto,Jupiter Ocean Grande Realty,None,None,Dianna Shpritz,"Leibowitz Realty Group, LLC./PBG",None,None
1,PAST SALE,2025-01-24,Condo/Co-op,140 SE 5th Ave #250,Boca Raton,FL,33432.0,1510000.0,2.0,2.0,...,-80.076619,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Marisela Cotilla,Pineapple Realty Advisores LLC,None,None,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC
2,PAST SALE,2025-01-24,Condo/Co-op,550 S Ocean Blvd #1006,Boca Raton,FL,33432.0,1450000.0,2.0,2.0,...,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None
3,PAST SALE,2025-01-21,Condo/Co-op,6 Little Pond Rd,Manalapan,FL,33462.0,1220000.0,2.0,2.0,...,-80.043908,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Michelle Newman,The Corcoran Group,None,None,Michelle Newman,The Corcoran Group,None,None
4,PAST SALE,2025-01-24,Condo/Co-op,424 Bay Colony Dr N,Juno Beach,FL,33408.0,1200000.0,3.0,2.5,...,-80.065793,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Crystal Condit,Waterfront Properties & Club C,None,None,Ashley Barbaree,Sutter & Nugent LLC,None,None
5,PAST SALE,2025-01-24,Condo/Co-op,3000 S Ocean Blvd #405,Boca Raton,FL,33432.0,1007500.0,2.0,2.0,...,-80.075232,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Cavan Dettman,"Compass Florida, LLC.",None,None
6,PAST SALE,2025-01-24,Condo/Co-op,242 Tresana Blvd #74,Jupiter,FL,33478.0,885000.0,3.0,2.5,...,-80.155641,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Allison Moore,Coldwell Banker Realty,None,None,Margot Matot,Platinum Properties/The Keyes,None,None
7,PAST SALE,2025-01-23,Condo/Co-op,3940 Back Bay 241 Dr #241,Jupiter,FL,33477.0,835000.0,3.0,2.0,...,-80.092082,https://www.redfin.com/FL/Jupiter/3940-Back-Ba...,Paul Stockall,Compass Florida LLC,Diane Hutchinson,Compass Florida LLC,Paul Stockall,Compass Florida LLC,Diane Hutchinson,Compass Florida LLC
8,PAST SALE,2025-01-24,Condo/Co-op,7233 Promenade Dr #701,Boca Raton,FL,33433.0,745000.0,3.0,2.5,...,-80.157913,https://www.redfin.com/FL/Boca-Raton/7233-Prom...,Edy Pyles,Casalina Realty Inc,None,None,None,None,None,None
9,PAST SALE,2025-01-21,Condo/Co-op,5214 Lake Catalina Dr N Unit C,Boca Raton,FL,33496.0,720000.0,3.0,2.0,...,-80.126160,https://www.redfin.com/FL/Boca-Raton/5214-Lake...,Michael Ledwitz,Engel & Volkers Boca Raton,Wendy Ledwitz,Engel & Volkers Boca Raton,Joy Wayne,Linda A. Gary Real Estate,None,None


## Current Week's Values

In [35]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

64


In [36]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$403,182


In [37]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$299


In [38]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$25,803,620


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [39]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
81
Input Previous Week Condo Average Sales Price:
65_700_000
Input Previous Week Condo Average PSF:
397
Input Previous Week Condo Sales total (ex: 198_000_000)
61_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [40]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [41]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [42]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [43]:
pd.set_option('display.max_columns',None)

In [44]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,Jupiter Ocean Grande 4 Condo,2490.0,NaN,2003.0,NaN,663.0,2033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Beaches MLS,RX-11028563,N,Y,26.918407,-80.066545,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Michael Montalto,Jupiter Ocean Grande Realty,None,None,Dianna Shpritz,"Leibowitz Realty Group, LLC./PBG",None,None,1,orange


In [45]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,Jupiter Ocean Grande 4 Condo,2490.0,NaN,2003.0,NaN,663.0,2033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Beaches MLS,RX-11028563,N,Y,26.918407,-80.066545,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Michael Montalto,Jupiter Ocean Grande Realty,None,None,Dianna Shpritz,"Leibowitz Realty Group, LLC./PBG",None,None,1,orange
1,PAST SALE,2025-01-24,Condo/Co-op,140 SE 5th Ave #250,Boca Raton,FL,33432.0,1510000.0,2.0,2.0,Mizner Court Condo,1630.0,NaN,1988.0,NaN,926.0,2747.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Beaches MLS,RX-11050978,N,Y,26.348269,-80.076619,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Marisela Cotilla,Pineapple Realty Advisores LLC,None,None,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,2,blue
2,PAST SALE,2025-01-24,Condo/Co-op,550 S Ocean Blvd #1006,Boca Raton,FL,33432.0,1450000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1001.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11031947,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue
3,PAST SALE,2025-01-21,Condo/Co-op,6 Little Pond Rd,Manalapan,FL,33462.0,1220000.0,2.0,2.0,Little Pond Park Residences Condo,1500.0,NaN,1977.0,NaN,813.0,1358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Beaches MLS,RX-10948720,N,Y,26.570376,-80.043908,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Michelle Newman,The Corcoran Group,None,None,Michelle Newman,The Corcoran Group,None,None,4,blue
4,PAST SALE,2025-01-24,Condo/Co-op,424 Bay Colony Dr N,Juno Beach,FL,33408.0,1200000.0,3.0,2.5,Juno Bay Colony,1839.0,NaN,2014.0,NaN,653.0,1557.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Beaches MLS,RX-11034043,N,Y,26.860900,-80.065793,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Crystal Condit,Waterfront Properties & Club C,None,None,Ashley Barbaree,Sutter & Nugent LLC,None,None,5,blue
5,PAST SALE,2025-01-24,Condo/Co-op,3000 S Ocean Blvd #405,Boca Raton,FL,33432.0,1007500.0,2.0,2.0,Three Thousand South,1382.0,NaN,1970.0,NaN,729.0,2211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Beaches MLS,RX-11024057,N,Y,26.321745,-80.075232,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Cavan Dettman,"Compass Florida, LLC.",None,None,6,blue
6,PAST SALE,2025-01-24,Condo/Co-op,242 Tresana Blvd #74,Jupiter,FL,33478.0,885000.0,3.0,2.5,Jupiter Country Club Condo Iii,2293.0,NaN,2019.0,NaN,386.0,1314.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Beaches MLS,RX-10959380,N,Y,26.926899,-80.155641,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Allison Moore,Coldwell Banker Realty,None,None,Margot Matot,Platinum Properties/The Keyes,None,None,7,blue
7,PAST SALE,2025-01-23,Condo/Co-op,3940 Back Bay 241 Dr #241,Jupiter,FL,33477.0,835000.0,3.0,2.0,West Bay AT Jonathans Landing Condo,1839.0,NaN,1990.0,NaN,454.0,1458.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/3940-Back-Ba...,Beaches MLS,RX-11044736,N,Y,26.924386,-80.092082,https://www.redfin.com/FL/Jupiter/3940-Back-Ba...,Paul Stockall,Compass Florida LLC,Diane Hutchinson,Compass Florida LLC,Paul Stockall,Compass Florida LLC,Diane Hutchinson,Compass Florida LLC,8,blue
8,PAST SALE,2025-01-24,Condo/Co-op,7233 Promenade Dr #701,Boca Raton,FL,33433.0,

In [47]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [48]:
m.save('index.html')

## Data snagger

In [49]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [50]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,Jupiter Ocean Grande 4 Condo,2490.0,NaN,2003.0,NaN,663.0,2033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Beaches MLS,RX-11028563,N,Y,26.918407,-80.066545,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Michael Montalto,Jupiter Ocean Grande Realty,None,None,Dianna Shpritz,"Leibowitz Realty Group, LLC./PBG",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [51]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_01272024


# CREATE TEMPLATE 

In [52]:
muni_set = set(df_top_ten['CITY'])

In [53]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [54]:
df_top_ten.reset_index(inplace=True,drop=True)

In [55]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [56]:
top_sale

'Jupiter Ocean Grande 4 Condo at 230 Ocean Grande Blvd #801 in Jupiter'

In [57]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [58]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,Jupiter Ocean Grande 4 Condo,2490.0,NaN,2003.0,NaN,663.0,2033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Beaches MLS,RX-11028563,N,Y,26.918407,-80.066545,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Michael Montalto,Jupiter Ocean Grande Realty,None,None,Dianna Shpritz,"Leibowitz Realty Group, LLC./PBG",None,None,1,orange,Jupiter Ocean Grande 4 Condo at 230 Ocean Gran...
1,PAST SALE,2025-01-24,Condo/Co-op,140 SE 5th Ave #250,Boca Raton,FL,33432.0,1510000.0,2.0,2.0,Mizner Court Condo,1630.0,NaN,1988.0,NaN,926.0,2747.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Beaches MLS,RX-11050978,N,Y,26.348269,-80.076619,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Marisela Cotilla,Pineapple Realty Advisores LLC,None,None,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,2,blue,Mizner Court Condo at 140 SE 5th Ave #250 in B...
2,PAST SALE,2025-01-24,Condo/Co-op,550 S Ocean Blvd #1006,Boca Raton,FL,33432.0,1450000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1001.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11031947,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue,Chalfonte Condo at 550 S Ocean Blvd #1006 in B...
3,PAST SALE,2025-01-21,Condo/Co-op,6 Little Pond Rd,Manalapan,FL,33462.0,1220000.0,2.0,2.0,Little Pond Park Residences Condo,1500.0,NaN,1977.0,NaN,813.0,1358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Beaches MLS,RX-10948720,N,Y,26.570376,-80.043908,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Michelle Newman,The Corcoran Group,None,None,Michelle Newman,The Corcoran Group,None,None,4,blue,Little Pond Park Residences Condo at 6 Little ...
4,PAST SALE,2025-01-24,Condo/Co-op,424 Bay Colony Dr N,Juno Beach,FL,33408.0,1200000.0,3.0,2.5,Juno Bay Colony,1839.0,NaN,2014.0,NaN,653.0,1557.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Beaches MLS,RX-11034043,N,Y,26.860900,-80.065793,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Crystal Condit,Waterfront Properties & Club C,None,None,Ashley Barbaree,Sutter & Nugent LLC,None,None,5,blue,Juno Bay Colony at 424 Bay Colony Dr N in Juno...
5,PAST SALE,2025-01-24,Condo/Co-op,3000 S Ocean Blvd #405,Boca Raton,FL,33432.0,1007500.0,2.0,2.0,Three Thousand South,1382.0,NaN,1970.0,NaN,729.0,2211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Beaches MLS,RX-11024057,N,Y,26.321745,-80.075232,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Cavan Dettman,"Compass Florida, LLC.",None,None,6,blue,Three Thousand South at 3000 S Ocean Blvd #405...
6,PAST SALE,2025-01-24,Condo/Co-op,242 Tresana Blvd #74,Jupiter,FL,33478.0,885000.0,3.0,2.5,Jupiter Country Club Condo Iii,2293.0,NaN,2019.0,NaN,386.0,1314.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Beaches MLS,RX-10959380,N,Y,26.926899,-80.155641,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Allison Moore,Coldwell Banker Realty,None,None,Margot Matot,Platinum Properties/The Keyes,None,None,7,blue,Jupiter Country Club Condo Iii at 242 Tresana ...
7,PAST SALE,2025-01-23,Condo/Co-op,3940 Back Bay 241 Dr #241,Jupiter,FL,33477.0,835000.0,3.0,2.0,West Bay AT Jonathans Landing Condo,1839.0,NaN,1990.0,NaN,454.0,1458.0,Sold,NaN,NaN,https://www.redfin

In [60]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Jupiter Ocean Grande 4 Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $720,000 to $1,650,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Jupiter, Manalapan, Juno Beach, Boca Raton

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 64 condo sales totaling $25,803,620 million from Jan. 19th to Jan. 26th. The previous week, brokers closed 81 condo sales totaling $61,300,000.

Last week’s units sold for an average of $403,182, lower than the $65,700,000 aver

In [61]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [62]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [63]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [64]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,Jupiter Ocean Grande 4 Condo,2490.0,NaN,2003.0,NaN,663.0,2033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Beaches MLS,RX-11028563,N,Y,26.918407,-80.066545,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Michael Montalto,Jupiter Ocean Grande Realty,None,None,Dianna Shpritz,"Leibowitz Realty Group, LLC./PBG",None,None,1,orange,Jupiter Ocean Grande 4 Condo at 230 Ocean Gran...
1,PAST SALE,2025-01-24,Condo/Co-op,140 SE 5th Ave #250,Boca Raton,FL,33432.0,1510000.0,2.0,2.0,Mizner Court Condo,1630.0,NaN,1988.0,NaN,926.0,2747.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Beaches MLS,RX-11050978,N,Y,26.348269,-80.076619,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Marisela Cotilla,Pineapple Realty Advisores LLC,None,None,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,2,blue,Mizner Court Condo at 140 SE 5th Ave #250 in B...
2,PAST SALE,2025-01-24,Condo/Co-op,550 S Ocean Blvd #1006,Boca Raton,FL,33432.0,1450000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1001.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11031947,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue,Chalfonte Condo at 550 S Ocean Blvd #1006 in B...
3,PAST SALE,2025-01-21,Condo/Co-op,6 Little Pond Rd,Manalapan,FL,33462.0,1220000.0,2.0,2.0,Little Pond Park Residences Condo,1500.0,NaN,1977.0,NaN,813.0,1358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Beaches MLS,RX-10948720,N,Y,26.570376,-80.043908,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Michelle Newman,The Corcoran Group,None,None,Michelle Newman,The Corcoran Group,None,None,4,blue,Little Pond Park Residences Condo at 6 Little ...
4,PAST SALE,2025-01-24,Condo/Co-op,424 Bay Colony Dr N,Juno Beach,FL,33408.0,1200000.0,3.0,2.5,Juno Bay Colony,1839.0,NaN,2014.0,NaN,653.0,1557.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Beaches MLS,RX-11034043,N,Y,26.860900,-80.065793,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Crystal Condit,Waterfront Properties & Club C,None,None,Ashley Barbaree,Sutter & Nugent LLC,None,None,5,blue,Juno Bay Colony at 424 Bay Colony Dr N in Juno...
5,PAST SALE,2025-01-24,Condo/Co-op,3000 S Ocean Blvd #405,Boca Raton,FL,33432.0,1007500.0,2.0,2.0,Three Thousand South,1382.0,NaN,1970.0,NaN,729.0,2211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Beaches MLS,RX-11024057,N,Y,26.321745,-80.075232,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Cavan Dettman,"Compass Florida, LLC.",None,None,6,blue,Three Thousand South at 3000 S Ocean Blvd #405...
6,PAST SALE,2025-01-24,Condo/Co-op,242 Tresana Blvd #74,Jupiter,FL,33478.0,885000.0,3.0,2.5,Jupiter Country Club Condo Iii,2293.0,NaN,2019.0,NaN,386.0,1314.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Beaches MLS,RX-10959380,N,Y,26.926899,-80.155641,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Allison Moore,Coldwell Banker Realty,None,None,Margot Matot,Platinum Properties/The Keyes,None,None,7,blue,Jupiter Country Club Condo Iii at 242 Tresana ...
7,PAST SALE,2025-01-23,Condo/Co-op,3940 Back Bay 241 Dr #241,Jupiter,FL,33477.0,835000.0,3.0,2.0,West Bay AT Jonathans Landing Condo,1839.0,NaN,1990.0,NaN,454.0,1458.0,Sold,NaN,NaN,https://www.redfin

In [65]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Jupiter Ocean Grande 4 Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $720,000 to $1,650,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Jupiter, Manalapan, Juno Beach, Boca Raton

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 64 condo sales totaling $25,803,620 million from Jan. 19th to Jan. 26th. The previous week, brokers closed 81 condo sales totaling $61,300,000.

Last week’s units sold for an average of $403,182, lower than the $65,700,000 aver

In [81]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Jupiter/230-Ocean-Grande-Blvd-33477/unit-801/home/172815762


In [82]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/140-SE-5th-Ave-33432/unit-250/home/42331175


In [83]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Boca-Raton/5214-Lake-Catalina-Dr-N-33496/unit-C/home/42237730


In [84]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Boca-Raton/7233-Promenade-Dr-33433/unit-701/home/42269547


In [85]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/550-S-Ocean-Blvd-33432/unit-1006/home/42326365


In [86]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Boca-Raton/7233-Promenade-Dr-33433/unit-701/home/42269547


In [87]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-21,Condo/Co-op,230 Ocean Grande Blvd #801,Jupiter,FL,33477.0,1650000.0,3.0,3.5,Jupiter Ocean Grande 4 Condo,2490.0,NaN,2003.0,NaN,663.0,2033.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Beaches MLS,RX-11028563,N,Y,26.918407,-80.066545,https://www.redfin.com/FL/Jupiter/230-Ocean-Gr...,Michael Montalto,Jupiter Ocean Grande Realty,None,None,Dianna Shpritz,"Leibowitz Realty Group, LLC./PBG",None,None,1,orange,Jupiter Ocean Grande 4 Condo at 230 Ocean Gran...
1,PAST SALE,2025-01-24,Condo/Co-op,140 SE 5th Ave #250,Boca Raton,FL,33432.0,1510000.0,2.0,2.0,Mizner Court Condo,1630.0,NaN,1988.0,NaN,926.0,2747.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Beaches MLS,RX-11050978,N,Y,26.348269,-80.076619,https://www.redfin.com/FL/Boca-Raton/140-SE-5t...,Marisela Cotilla,Pineapple Realty Advisores LLC,None,None,Rachelle Hirt-Beresh,Compass Florida LLC,Scott Gerow,Compass Florida LLC,2,blue,Mizner Court Condo at 140 SE 5th Ave #250 in B...
2,PAST SALE,2025-01-24,Condo/Co-op,550 S Ocean Blvd #1006,Boca Raton,FL,33432.0,1450000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1001.0,1569.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11031947,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,3,blue,Chalfonte Condo at 550 S Ocean Blvd #1006 in B...
3,PAST SALE,2025-01-21,Condo/Co-op,6 Little Pond Rd,Manalapan,FL,33462.0,1220000.0,2.0,2.0,Little Pond Park Residences Condo,1500.0,NaN,1977.0,NaN,813.0,1358.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Beaches MLS,RX-10948720,N,Y,26.570376,-80.043908,https://www.redfin.com/FL/Lantana/6-Little-Pon...,Michelle Newman,The Corcoran Group,None,None,Michelle Newman,The Corcoran Group,None,None,4,blue,Little Pond Park Residences Condo at 6 Little ...
4,PAST SALE,2025-01-24,Condo/Co-op,424 Bay Colony Dr N,Juno Beach,FL,33408.0,1200000.0,3.0,2.5,Juno Bay Colony,1839.0,NaN,2014.0,NaN,653.0,1557.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Beaches MLS,RX-11034043,N,Y,26.860900,-80.065793,https://www.redfin.com/FL/Juno-Beach/424-Bay-C...,Crystal Condit,Waterfront Properties & Club C,None,None,Ashley Barbaree,Sutter & Nugent LLC,None,None,5,blue,Juno Bay Colony at 424 Bay Colony Dr N in Juno...
5,PAST SALE,2025-01-24,Condo/Co-op,3000 S Ocean Blvd #405,Boca Raton,FL,33432.0,1007500.0,2.0,2.0,Three Thousand South,1382.0,NaN,1970.0,NaN,729.0,2211.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Beaches MLS,RX-11024057,N,Y,26.321745,-80.075232,https://www.redfin.com/FL/Boca-Raton/3000-S-Oc...,Gregory Lynn,Lynn Realty Group,Charli Lynn,Lynn Realty Group,Cavan Dettman,"Compass Florida, LLC.",None,None,6,blue,Three Thousand South at 3000 S Ocean Blvd #405...
6,PAST SALE,2025-01-24,Condo/Co-op,242 Tresana Blvd #74,Jupiter,FL,33478.0,885000.0,3.0,2.5,Jupiter Country Club Condo Iii,2293.0,NaN,2019.0,NaN,386.0,1314.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Beaches MLS,RX-10959380,N,Y,26.926899,-80.155641,https://www.redfin.com/FL/Jupiter/242-Tresana-...,Allison Moore,Coldwell Banker Realty,None,None,Margot Matot,Platinum Properties/The Keyes,None,None,7,blue,Jupiter Country Club Condo Iii at 242 Tresana ...
7,PAST SALE,2025-01-23,Condo/Co-op,3940 Back Bay 241 Dr #241,Jupiter,FL,33477.0,835000.0,3.0,2.0,West Bay AT Jonathans Landing Condo,1839.0,NaN,1990.0,NaN,454.0,1458.0,Sold,NaN,NaN,https://www.redfin

## Time on Market Calculator

In [80]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 10, 15) ## List (Earlier) date
date2 = datetime(2025, 1, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

98


## Clean CSV for Datawrapper Chart

In [88]:
chart_df = df_top_ten

In [89]:
chart_df = chart_df.fillna(" ")

In [90]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [91]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [92]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [93]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [94]:
chart_df['$/SQUARE FEET'].astype(float)

0     663.0
1     926.0
2    1001.0
3     813.0
4     653.0
5     729.0
6     386.0
7     454.0
8     235.0
9     396.0
Name: $/SQUARE FEET, dtype: float64

In [95]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [96]:
csv_date_string = today.strftime("%m_%d_%Y")

In [97]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [98]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-21,230 Ocean Grande Blvd #801 in Jupiter,"$1,650,000",3,3.5,"2,490",2003,$663,Beaches MLS,RX-11028563,Michael Montalto Jupiter Ocean Grande Realty,"Dianna Shpritz Leibowitz Realty Group, LLC./PB..."
1,2025-01-24,140 SE 5th Ave #250 in Boca Raton,"$1,510,000",2,2.0,"1,630",1988,$926,Beaches MLS,RX-11050978,Marisela Cotilla Pineapple Realty Advisores LL...,Rachelle Hirt-Beresh Compass Florida LLC Scott...
2,2025-01-24,550 S Ocean Blvd #1006 in Boca Raton,"$1,450,000",2,2.0,"1,448",1974,"$1,001",Beaches MLS,RX-11031947,Ana Londono Lang Realty/BR,Ana Londono Lang Realty/BR
3,2025-01-21,6 Little Pond Rd in Manalapan,"$1,220,000",2,2.0,"1,500",1977,$813,Beaches MLS,RX-10948720,Michelle Newman The Corcoran Group,Michelle Newman The Corcoran Group
4,2025-01-24,424 Bay Colony Dr N in Juno Beach,"$1,200,000",3,2.5,"1,839",2014,$653,Beaches MLS,RX-11034043,Crystal Condit Waterfront Properties & Club C,Ashley Barbaree Sutter & Nugent LLC
5,2025-01-24,3000 S Ocean Blvd #405 in Boca Raton,"$1,007,500",2,2.0,"1,382",1970,$729,Beaches MLS,RX-11024057,Gregory Lynn Lynn Realty Group Charli Lynn Lyn...,"Cavan Dettman Compass Florida, LLC."
6,2025-01-24,242 Tresana Blvd #74 in Jupiter,"$885,000",3,2.5,"2,293",2019,$386,Beaches MLS,RX-10959380,Allison Moore Coldwell Banker Realty,Margot Matot Platinum Properties/The Keyes
7,2025-01-23,3940 Back Bay 241 Dr #241 in Jupiter,"$835,000",3,2.0,"1,839",1990,$454,Beaches MLS,RX-11044736,Paul Stockall Compass Florida LLC Diane Hutchi...,Paul Stockall Compass Florida LLC Diane Hutchi...
8,2025-01-24,7233 Promenade Dr #701 in Boca Raton,"$745,000",3,2.5,"3,165",1986,$235,MARMLS,A11697582,Edy Pyles Casalina Realty Inc,
9,2025-01-21,5214 Lake Catalina Dr N Unit C in Boca Raton,"$720,000",3,2.0,"1,819",1989,$396,Beaches MLS,RX-11014143,Michael Ledwitz Engel & Volkers Boca Raton Wen...,Joy Wayne Linda A. Gary Real Estate
